In [6]:
#install quandl using this command 
pip install quandl

Note: you may need to restart the kernel to use updated packages.


In [45]:
import pandas as pd
import quandl
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

In [8]:
#get the google stocks 
df = quandl.get('WIKI/GOOGL')

In [10]:
#print the head to see what we are working with 
print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [11]:
#we are going to adjust the df to what we need by making a list of columns we need
df = df[['Adj. Open','Adj. High','Adj. Low','Adj. Close', 'Adj. Volume']]
#lets see what we have
print(df.head())

            Adj. Open  Adj. High   Adj. Low  Adj. Close  Adj. Volume
Date                                                                
2004-08-19  50.159839  52.191109  48.128568   50.322842   44659000.0
2004-08-20  50.661387  54.708881  50.405597   54.322689   22834300.0
2004-08-23  55.551482  56.915693  54.693835   54.869377   18256100.0
2004-08-24  55.792225  55.972783  51.945350   52.597363   15247300.0
2004-08-25  52.542193  54.167209  52.100830   53.164113    9188600.0


In [15]:
#now lets see what is the % change that will be calculated as high-old/old*100 and save the result in a new column

df['Hl_Pct'] = (df['Adj. High']-df['Adj. Close'])/df['Adj. Close']*100

#lets do the same for change which is going to be new-old/old*100

df['Pct_change'] = (df['Adj. Close']-df['Adj. Open'])/df['Adj. Open']*100


In [28]:
#name a dataset of the only colums we need now 

df = df[['Adj. Close','Hl_Pct','Pct_change','Adj. Volume']]

#lets see what we have
df.head()

,Adj. Close,Hl_Pct,Pct_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0


In [30]:
# lets define a string value of the adj. Close column name to use later

forecast_colm = 'Adj. Close'

#in machine learning you cant work with NaN data, so we need to replace it instead of getting rid of it 

df.fillna(-99999, inplace=True)

In [35]:
#now lets create regression

#rounding up to the closest integer and converting to int , Forecast_out is preficting 1 percent of the df
import math

forecast_out = int(math.ceil(0.01*len(df)))

In [36]:
# now we will create the label which will be the forecast_out numbers of column shift of the adj.close price

df['label'] = df[forecast_colm].shift(-forecast_out)

#lets see what we got 

df.head()

,Adj. Close,Hl_Pct,Pct_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,67.739104
2004-08-20,54.322689,0.710922,7.227007,22834300.0,69.399229
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,68.752232
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,69.639972
2004-08-25,53.164113,1.886792,1.183658,9188600.0,69.078238


In [39]:
#check the endpoint

df.dropna(inplace=True)
df.tail()

,Adj. Close,Hl_Pct,Pct_change,Adj. Volume,label
Date,,,,,
2016-09-22,815.95,0.381151,0.734568,1759290.0,781.10
2016-09-23,814.96,0.250319,-0.022082,1411673.0,802.03
2016-09-26,802.65,0.925684,-0.885382,1472732.0,811.98
2016-09-27,810.73,0.340434,1.109961,1367271.0,805.59
2016-09-28,810.06,0.023455,0.743707,1470280.0,780.29


In [65]:
#now we have our model ready to predict some data

X = np.array(df.drop(['label'],1))
y = np.array(df['label'])

#lets see what we got
X

array([[ 5.03228418e+01,  3.71256291e+00,  3.24967503e-01,
         4.46590000e+07],
       [ 5.43226889e+01,  7.10922353e-01,  7.22700723e+00,
         2.28343000e+07],
       [ 5.48693765e+01,  3.72943327e+00, -1.22788010e+00,
         1.82561000e+07],
       ...,
       [ 8.02650000e+02,  9.25683673e-01, -8.85381937e-01,
         1.47273200e+06],
       [ 8.10730000e+02,  3.40433930e-01,  1.10996096e+00,
         1.36727100e+06],
       [ 8.10060000e+02,  2.34550527e-02,  7.43707094e-01,
         1.47028000e+06]])

In [66]:
y

array([ 67.73910413,  69.39922877,  68.75223155, ..., 811.98      ,
       805.59      , 780.29      ])

In [67]:
X = preprocessing.scale(X)

In [70]:
#lets make sure we have the correct lengths here

print(len(X),len(y))

3050 3050


In [77]:
# now we need to make a training and testing set using 20 percent of the data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.2)

In [88]:
lr_clf = LinearRegression()

#fit is synonym of train and score is synonym for test

#we can also make it faster by defining n_jobs
#lt_clf = LinearRegression(n_jobs=10) or n_jobs=-1 to make it as fast as possible

lr_clf.fit(X_train, y_train)
accuracy = lr_clf.score(X_test, y_test)

In [89]:
# now lets check the accuracy of our data 

print("Accuracy comes up to with Linear Regression : ",accuracy )

Accuracy comes up to with Linear Regression :  0.9662397580696427


In [82]:
#lets change the algorithm to svm 

svm_slf = svm.SVR()

svm_slf.fit(X_train, y_train)
accuracy = svm_slf.score(X_test, y_test)

print("Accuracy comes up to with SVR : ",accuracy)

Accuracy comes up to with SVR :  0.8053419243294085
